### IMPORTACIÓN LIBRERIAS

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from sklearn import linear_model, metrics, model_selection
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

### LECTURA DE CSV Y PROCESAMIENTO DATATIME DE FECHAS

In [2]:
df = pd.read_csv("DF Precisión, Temperaturas y OF.csv")
df["Fecha/hora inicio"] = pd.to_datetime(df["Fecha/hora inicio"])
df["Fecha/hora fin"] = pd.to_datetime(df["Fecha/hora fin"])

### LECTURA DE CSV Y LIMPIEZA DE LOS DATOS

In [5]:
dd = pd.read_csv("OF 123456 v03.csv", encoding="Latin1")
dd["Cantidad entregada"] = dd["Cantidad entregada"].astype(str).str.replace(",", ".")
dd['Cantidad entregada'] = dd['Cantidad entregada'].str.replace("0", "")
dd["Lote"]= dd["Lote"].str.replace("/","")
df["Cantidad entregada"] = dd["Cantidad entregada"] 

### CREACIÓN DE NUEVAS VARIABLES SACANDO EL PROMEDIO Y MULTIPLICANDO VARIABLES PARA VER EL IMPACTO DE LA COMBINACIÓN

In [6]:
df['Indice_condiciones_almacen'] = (df['Temperatura almacén principal (inicio)'] + df['Temperatura almacén producción (inicio)']) / 2
df['Interaccion_temp_hum'] = df['Temperatura biorreactores (inicio)'] * df['Humedad biorreactores (inicio)']

### CONCATENAR MEDIAS CON EL CSV QUE TENEMOS TODAS LAS VARIABLES TRANSFORMADAS

In [7]:
medias = pd.read_csv("resultado_media_temperaturas (1).csv")
df_test = pd.read_csv("DF test con todo.csv")
df_test = pd.concat([df_test, medias], axis=1)

### CONCATENADO DE NUEVAS VARIABLES Y DATASET

In [8]:
medias_train = pd.read_csv("resultado_media_biorreactores_centrifugas.csv")
df = pd.concat([df, medias_train], axis=1)

### SELECCIÓN DE VARIABLES PARA EL MODELO

In [9]:
df_test = df_test[['LOTE',
 'Volumen de inóculo utilizado',
 'Viabilidad final cultivo',
 'Centrifugación 1 turbidez',
 'Centrifugación 2 turbidez',
 'Humedad almacén producción (inicio)',
 'Humedad almacén producción (fin)',
 'Humedad biorreactores (inicio)',
 'Temperatura biorreactores (fin)',
 'Temperatura centrifugas (inicio)',
 'Cantidad entregada',
 'Indice_condiciones_almacen',
 'Interaccion_temp_hum',
 'Humedad almacén principal (media)',
 'Temperatura biorreactores (media)',
 'Producto 1']]

In [10]:
df = df[['LOTE',
 'Volumen de inóculo utilizado',
 'Viabilidad final cultivo',
 'Centrifugación 1 turbidez',
 'Centrifugación 2 turbidez',
 'Humedad almacén producción (inicio)',
 'Humedad almacén producción (fin)',
 'Humedad biorreactores (inicio)',
 'Temperatura biorreactores (fin)',
 'Temperatura centrifugas (inicio)',
 'Cantidad entregada',
 'Indice_condiciones_almacen',
 'Interaccion_temp_hum',
 'Humedad almacén principal (media)',
 'Temperatura biorreactores (media)',
 'Producto 1']]

### SACAR MEDIANAS DE LOS OUT LAYERS

In [11]:
Q1 = df['Centrifugación 1 turbidez'].quantile(0.25)
Q3 = df['Centrifugación 1 turbidez'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df['Centrifugación 1 turbidez'] = df['Centrifugación 1 turbidez'].apply(
    lambda x: df['Centrifugación 1 turbidez'].median() if x < lower_bound or x > upper_bound else x
)

Q1 = df['Centrifugación 2 turbidez'].quantile(0.25)
Q3 = df['Centrifugación 2 turbidez'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df['Centrifugación 2 turbidez'] = df['Centrifugación 2 turbidez'].apply(
    lambda x: df['Centrifugación 2 turbidez'].median() if x < lower_bound or x > upper_bound else x
)

Q1 = df['Temperatura biorreactores (fin)'].quantile(0.25)
Q3 = df['Temperatura biorreactores (fin)'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df['Temperatura biorreactores (fin)'] = df['Temperatura biorreactores (fin)'].apply(
    lambda x: df['Temperatura biorreactores (fin)'].median() if x < lower_bound or x > upper_bound else x)

### FUNCIÓN DE ENTRENAMIENTO DE MODELO CON PARAMETROS, ESCALADO DE LOS DATOS Y PARTICIÓN DE TEST TRAIN

In [12]:
def train_and_evaluate_models(df, target_column):
    # Definir X e y
    X = df.drop(columns=[target_column])
    y = df[target_column]

    # Dividir el conjunto de datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

    # Escalar los datos
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convertir a DataFrame para mantener los nombres de columnas
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

    # Definir el modelo XGBRegressor con los parámetros especificados
    model = XGBRegressor(
        max_depth=2,
        learning_rate=0.01,
        n_estimators=550,
        subsample=0.8,
        colsample_bytree=0.8,
        alpha=0.1,
        reg_lambda=1,
        gamma=1,
        random_state=42
    )

    # Entrenar el modelo
    print("Training XGBRegressor...")
    model.fit(X_train_scaled, y_train)

    # Evaluar en conjunto de prueba
    y_pred = model.predict(X_test_scaled)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    # Mostrar resultados
    print(f"XGBRegressor - Test R²: {r2}, MSE: {mse}, RMSE: {rmse}")

    # Convertir los resultados a DataFrame
    results_df = pd.DataFrame([{
        'Model': 'XGBRegressor',
        'Train R² (CV)': model.score(X_train_scaled, y_train),
        'Test MSE': mse,
        'Test RMSE': rmse,
        'Test R²': r2
    }])

    return model, scaler, results_df



def predict_with_model(trained_model, scaler, predict_df):
    """
    Genera predicciones utilizando un modelo entrenado y un escalador.

    Args:
        trained_model: Modelo entrenado.
        scaler: Escalador ajustado.
        predict_df (pd.DataFrame): DataFrame con las características para predecir.

    Returns:
        pd.Series: Predicciones del modelo.
    """
    # Escalar datos
    X_scaled = scaler.transform(predict_df)
    predictions = trained_model.predict(X_scaled)
    return pd.Series(predictions, name='Predicción')

### LLAMAR A LA FUNCIÓN Y DARLE LOS PARAMETROS

In [13]:
best_model, scaler, results_df = train_and_evaluate_models(df, target_column='Producto 1')

Training XGBRegressor...
XGBRegressor - Test R²: 0.42246916100928533, MSE: 42117.63026818626, RMSE: 205.22580312471982


In [14]:
df_test.drop(columns=['Producto 1'], inplace=True)

### HACER PREDICCIONES DE TEST

In [15]:
predictions = predict_with_model(best_model, scaler, df_test)
print(predictions)

0     1454.314941
1     1617.307861
2     1482.116821
3     1374.013550
4     1250.369629
5     1372.709839
6     1505.699341
7     1468.254028
8     1259.354126
9      965.291992
10    1409.793823
11    1460.327148
12    1252.439087
13    1446.718994
14    1503.704468
15    1555.890747
16    1374.065186
17    1250.210083
18    1291.697998
19    1235.219849
20    1292.248047
21    1308.480957
22    1320.460938
23    1367.539795
24    1109.661499
25    1228.758301
26    1325.146484
27    1367.583008
28    1373.256348
29    1369.824219
30    1578.615723
31    1428.766968
32    1261.742188
33    1265.631592
34    1424.238770
35    1414.708008
36    1416.445801
37    1471.947754
38    1210.987915
39    1132.222168
40    1465.848389
41    1249.028442
42    1179.102295
43    1460.562256
44    1497.570923
45    1464.879150
46    1550.379395
47    1429.218384
48    1241.954102
49    1251.831787
50    1523.206787
51    1475.573364
52    1326.424805
53    1460.611572
54    1474.858521
Name: Pred

In [16]:
df_lote = df_test["LOTE"]
df_ent =predictions.to_frame()
df_concat = pd.concat([df_lote,df_ent], axis=1)

In [17]:
df_concat.to_csv("CVC DATA_UH2024.txt",index=False, header=False, sep='|')